<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Complaints Classification using Vantage and Amazon Bedrock
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial;'><b>Introduction:</b></p>

<p style = 'font-size:16px;font-family:Arial;'>Revolutionize customer complaint resolution with our pioneering solution, which seamlessly integrates the capabilities of <b>Teradata Vantage</b> and <b>Amazon Bedrock</b> model as LLM. This powerful synergy enables businesses to classify customer complaints with unmatched precision and speed, allowing them to swiftly identify and address concerns, thereby elevating overall customer satisfaction and loyalty.</p>

<p style = 'font-size:16px;font-family:Arial;'><b>Key Features:</b></p>
<ul style = 'font-size:16px;font-family:Arial;'>
    <li><b>Automated Classification:</b> Our AI-driven model categorizes complaints into predefined categories, ensuring consistency and reducing manual effort.</li>
    <li><b>Contextual Understanding:</b> The system comprehends the nuances of customer feedback, capturing subtle differences in tone and language.</li>
    <li><b>Real-time Insights:</b> Generate instant reports and analytics to identify trends, patterns, and areas for improvement.</li>
</ul>


<p style = 'font-size:16px;font-family:Arial;'><b>Benefits:</b></p>
<ul style = 'font-size:16px;font-family:Arial;'>
    <li><b>Enhanced Customer Experience:</b> Swiftly address customer concerns, fostering trust and loyalty.</li>
    <li><b>Improved Operational Efficiency:</b> Reduce manual processing time, allowing teams to focus on high-value tasks.</li>
    <li><b>Data-Driven Decision Making: </b> Make informed decisions with actionable insights from complaint data.</li>
</ul>

<p style = 'font-size:16px;font-family:Arial;'>Experience the transformative power of Generative AI in complaints classification.</p>

<p style = 'font-size:16px;font-family:Arial;'><b>Steps in the analysis:</b></p>
<ol style = 'font-size:16px;font-family:Arial;'>
    <li>Connect to Vantage</li>
    <li>Configuring AWS CLI</li>
    <li>Classify Compalints</li>
    <li>Cleanup</li>
</ol>

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial;'><b>Downloading and installing additional software needed</b>

In [ ]:
%%capture 

!pip install --upgrade -r requirements.txt --quiet

<div class="alert alert-block alert-info">
<p style = 'font-size:16px;font-family:Arial;'><b>Note: </b><i>Please restart the kernel after executing these two lines. The simplest way to restart the Kernel is by typing zero zero: <b> 0 0</b></i></p>

<hr style="height:2px;border:none;">
<p style = 'font-size:16px;font-family:Arial;'>Here, we import the required libraries, set environment variables and environment paths (if required).</p>

In [ ]:
import numpy as np
import pandas as pd
import timeit
import boto3
from tqdm import tqdm
from teradataml import *
import plotly.express as px
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import plotly.subplots as subplots

display.max_rows = 5
pd.set_option('display.max_colwidth', None)
from IPython.display import display, Markdown

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial;'>1. Connect to Vantage</b>
<p style = 'font-size:16px;font-family:Arial;'>We will be prompted to provide the password. We will enter the password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)
execute_sql('''SET query_band='DEMO=Text_Classification.ipynb;' UPDATE FOR SESSION;''')

<p style = 'font-size:16px;font-family:Arial;'>Begin running steps with Shift + Enter keys. </p>

<p style = 'font-size:20px;font-family:Arial;'><b>Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial;'>We have provided data for this demo on cloud storage. We have the option of either running the demo using foreign tables to access the data without using any storage on our environment or downloading the data to local storage, which may yield somewhat faster execution. However, we need to consider available storage. There are two statements in the following cell, and one is commented out. We may switch which mode we choose by changing the comment string.</p>

In [ ]:
# %run -i ../run_procedure.py "call get_data('DEMO_ComplaintAnalysis_cloud');"        # Takes 1 minute
%run -i ../run_procedure.py "call get_data('DEMO_ComplaintAnalysis_local');"        # Takes 2 minutes

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial;'>2. Configuring AWS CLI</b>
<p style = 'font-size:16px;font-family:Arial;'>The following cell will prompt us for the following information:</p>
<ol style = 'font-size:16px;font-family:Arial;'>
<li><b>aws_access_key_id</b>: Enter your AWS access key ID</li>
<li><b>aws_secret_access_key</b>: Enter your AWS secret access key</li>
<li><b>region name</b>: Enter the AWS region you want to configure (e.g., us-east-1)</li>
<ol>

<div class="alert alert-block alert-info">
<p style = 'font-size:16px;font-family:Arial;'><b>Note: </b><i>If the AWS CLI commands below fail to execute or encounter issues, you may proceed directly to Section 2.1, where the same configuration is performed using Boto3. </i></p> 

In [ ]:
# def configure_aws():
#     print("configure the AWS CLI")
#     # enter the access_key/secret_key
#     access_key = getpass.getpass("aws_access_key_id ")
#     secret_key = getpass.getpass("aws_secret_access_key ")
#     region_name = getpass.getpass("region name")

#     #set to the env
#     !aws configure set aws_access_key_id {access_key}
#     !aws configure set aws_secret_access_key {secret_key}
#     !aws configure set default.region {region_name}

In [ ]:
# does_access_key_exists = !aws configure get aws_access_key_id

# if len(does_access_key_exists) == 0:
#     configure_aws()

In [ ]:
# !aws configure list

<hr style="height:1px;border:none;">
<b style = 'font-size:18px;font-family:Arial;'>2.1 Initialize the Bedrock Model</b>
<ul style = 'font-size:16px;font-family:Arial;'>
<li>The code below initializes a Boto3 client for the “bedrock-runtime” service.</li>
<li>The model can be used for natural language generation tasks.</li>
<ul>

In [ ]:
session = boto3.Session()
credentials = session.get_credentials()

# Create a Bedrock Runtime client in the AWS Region of your choice.
client = boto3.client(
    "bedrock-runtime",
    region_name="us-east-1",
    aws_access_key_id = getpass.getpass("aws_access_key_id:"),
    aws_secret_access_key = getpass.getpass("aws_secret_access_key:"),
    aws_session_token = getpass.getpass("Enter AWS Session Token: ")
    
)

<ul style = 'font-size:16px;font-family:Arial;'>
<li>The code below tests a Boto3 client connection to the Amazon Bedrock “bedrock-runtime” service by sending a sample message to a chatbot model.</li>

In [ ]:
from botocore.exceptions import ClientError

try:
    response = client.converse(
        modelId="amazon.nova-lite-v1:0",
        messages=[{
            "role": "user",
            "content": [{"text": "Hello"}]
        }],
        inferenceConfig={"maxTokens": 10}
    )
    print("Test call successful! Response:")
    print(response)
except ClientError as e:
    print(f"Error testing client: {e}")

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial;'>3. Classify Complaints</b>
<p style = 'font-size:16px;font-family:Arial;'>We'll use a sample of the data to classify complaints</p>

In [ ]:
tdf = DataFrame(in_schema('DEMO_ComplaintAnalysis', 'Consumer_Complaints'))
tdf = tdf.assign(id = tdf.complaint_id).drop('complaint_id', axis = 1)
tdf

In [ ]:
def classify_review(prompt):
    request_body = {
        "system": [
            {"text": "You are a review categorization agent"}
        ],
        "messages": [
            {
                "role": "user",
                "content": [{"text": prompt}]
            },
        ],
        "inferenceConfig": {
            "maxTokens": 300,
            "topP": 0.9,
            "topK": 20,
            "temperature": 0.7,
        }
    }
    
   # Invoke the model and extract the response body.
    response = client.invoke_model(
        modelId="amazon.nova-lite-v1:0",
        body=json.dumps(request_body)
    )
    model_response = json.loads(response["body"].read())
    return(model_response)

In [ ]:
df = tdf.to_pandas(num_rows = 20)
df['Prediction'] = ""
df['Reasoning with Chain of Thought'] = ""

In [ ]:
for i in tqdm(range(len(df))):
    try:
        prompt = f'''
        User prompt:
        The following is text from a review:

        “{df['consumer_complaint_narrative'][i]}”

        Give me reasoning as well as Category for this review

        Instructions for Reasoning:
        - Give me Reasoning in detail
        - Only one sentence reasoning
        Instructions for Category:
        - The review falls into one of the following categories: Complaint, Non-Complaint
        - Select one category from the given ones

        My output comes in the format:
        Category: 
        Reasoning: 
        '''
        response = classify_review(prompt)
        text = response['output']['message']['content'][0]['text']
        category = re.search('Category:(.*)', text).group(1)
        if category == "":
            category = re.search('Category:\n(.*)', text).group(1)
        reasoning = re.search('Reasoning:(.*)', text).group(1)
        if reasoning == "":
            reasoning = re.search('Reasoning:\n(.*)', text).group(1)
        df['Prediction'][i] = category.strip()
        df['Reasoning with Chain of Thought'][i] = reasoning.strip()
    except:
        pass

In [ ]:
df[['id', 'consumer_complaint_narrative', 'Prediction', 'Reasoning with Chain of Thought']].head(5)

<hr style='height:1px;border:none;'>
<p style = 'font-size:18px;font-family:Arial;'><b>3.1 Consumer Complaints Prediction vs Occurrences</b></p>

<p style = 'font-size:16px;font-family:Arial;'>A graph illustrating the relationship between consumer complaints prediction and the number of occurrences. This visual representation helps identify trends, patterns, and areas for improvement, enabling data-driven decision making.</p>

In [ ]:
from collections import Counter
data = Counter(df['Prediction'])

# Convert Counter data to DataFrame
viz_df = pd.DataFrame.from_dict(data, orient='index', columns=['Count']).reset_index()

# Rename columns
viz_df.columns = ['Prediction', 'Count']

# Create bar graph using Plotly Express
fig = px.bar(viz_df, x='Prediction', y='Count', color='Prediction',
             labels={'Count': 'Number of Occurrences', 'Prediction': 'Prediction'})

# Show the plot
fig.show(renderer="notebook")

<hr style='height:1px;border:none;'>
<p style = 'font-size:18px;font-family:Arial;'><b>3.2 Word Cloud for Consumer Complaints Prediction</b></p>

<p style = 'font-size:16px;font-family:Arial;'>A visual representation of <b>consumer complaints prediction</b>, highlighting the most frequent words and pain points in customer feedback. This word cloud helps identify trends, sentiment, and areas for improvement, enabling data-driven decision making.</p>

In [ ]:
def display_helper(msg):
    return display(Markdown(
        f"""<div class="alert alert-block alert-info">
        <p style = 'font-size:16px;font-family:Arial;'><b>Note: </b>
        <i>{msg}</i></p>"""))

In [ ]:
complaint = df[df['Prediction'] == 'Complaint']
complaint_text = ' '.join(complaint['consumer_complaint_narrative'])

# Replace 'X' with blank space
modified_string = complaint_text.replace('X', '')

if len(modified_string):
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(modified_string)

    # Display the word cloud
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title("Complaints")
    plt.tight_layout()
    plt.axis("off")
    plt.show()
else:
    display_helper("""We included complaint and non-complaint options for completeness. 
    It's possible that our sample dataset doesn't contain any actual complaints.""")

<hr style='height:1px;border:none;'>
<p style = 'font-size:18px;font-family:Arial;'><b>3.3 Word Cloud for Non-Complaints Prediction</b></p>

<p style = 'font-size:16px;font-family:Arial;'>A visual representation of <b>non-complaints prediction</b>, highlighting the most frequent words and positive sentiments in customer feedback. This word cloud helps identify trends, sentiment, and areas of satisfaction, enabling data-driven decision making.</p>

In [ ]:
non_complaint = df[df['Prediction'] == 'Non-Complaint']
non_complaint_text = ' '.join(non_complaint['consumer_complaint_narrative'])

# Replace 'X' with blank space
modified_string = non_complaint_text.replace('X', '')

if len(modified_string):
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(modified_string)

    # Display the word cloud
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title("Non-Complaints")
    plt.tight_layout()
    plt.axis("off")
    plt.show()
else:
    display_helper("""We included both complaint and non-complaint options for completeness. 
          But since this is a complaints dataset, we don't expect to see any <b>non-complaints</b>.""")

<p style = 'font-size:16px;font-family:Arial;'>Now the results can be saved back to Vantage.</p>

In [ ]:
copy_to_sql(df = df, table_name = 'reviews', if_exists = 'replace')

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial;'>4. Cleanup</b>

<p style = 'font-size:18px;font-family:Arial;'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial;'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('DEMO_ComplaintAnalysis');"        # Takes 10 seconds

In [ ]:
remove_context()

<hr style="height:1px;border:none;">
<b style = 'font-size:18px;font-family:Arial;'>Dataset:</b>
<br>
<br>
<p style='font-size: 16px; font-family: Arial; color: #00233C;'>The dataset is sourced from <a href='https://www.consumerfinance.gov/data-research/consumer-complaints/'>Consumer Financial Protection Bureau</a></p>

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2024, 2025. All Rights Reserved
        </div>
    </div>
</footer>